In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [2]:
df = pd.read_csv("Churn_Modelling.csv")
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [3]:
df.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


# Remove Unnecessary Columns
Irrelevant Columns : RowNumber, CustomerId, Surname

In [4]:
X.drop(['RowNumber','CustomerId','Surname'], axis=1, inplace=True)

# Encoding Categorical Data

In [5]:
#LabelEncoding Gender Column
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
X['Gender'] = encoder.fit_transform(X['Gender'])

In [6]:
#Encoding Geography Column into a Binary Matrix
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
geo_encoded = pd.DataFrame(encoder.fit_transform(X['Geography']), columns=['Geo_1','Geo_2','Geo_3'])

In [7]:
#Adding Binary Matrix to X
X = pd.concat([X, geo_encoded], axis=1)

In [8]:
X.drop('Geography', axis=1, inplace=True)

# Train-Test-Split

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Feature Scaling on all Features (Very Important for Deep Learning)

In [10]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

# Building the NN 

In [15]:
#Initialising the ANN
#ANN is a sequence of layers
ann = tf.keras.models.Sequential()

#Adding Layers
ann.add(tf.keras.layers.Dense(units = 6,
                              activation = 'relu'))
ann.add(tf.keras.layers.Dense(units = 8,
                              activation = 'relu'))
ann.add(tf.keras.layers.Dense(units = 8,
                              activation = 'relu'))
#Sigmoid as it is a binary classification, it gives a probability of each class
ann.add(tf.keras.layers.Dense(units = 1,
                              activation = 'sigmoid'))

# Compiling the NN

In [16]:
#Binary -> Loss Function must be binary_crossentropy
#If Non-Binary Classification -> Loss Function = Categorical_crossentropy -> Last Layer = Softmax
ann.compile(optimizer = 'adam',
            loss = 'binary_crossentropy',
            metrics = ['accuracy'])

# Training the NN

In [17]:
#Batch_size -> Denotes whether you want batch learning (32 is the default number)
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 0s 429us/step - loss: 0.5421 - accuracy: 0.7558
Epoch 2/100
250/250 [==============================] - 0s 422us/step - loss: 0.4583 - accuracy: 0.7947
Epoch 3/100
250/250 [==============================] - 0s 416us/step - loss: 0.4347 - accuracy: 0.8075
Epoch 4/100
250/250 [==============================] - 0s 429us/step - loss: 0.4225 - accuracy: 0.8151
Epoch 5/100
250/250 [==============================] - 0s 425us/step - loss: 0.4132 - accuracy: 0.8224
Epoch 6/100
250/250 [==============================] - 0s 430us/step - loss: 0.4012 - accuracy: 0.8294
Epoch 7/100
250/250 [==============================] - 0s 426us/step - loss: 0.3841 - accuracy: 0.8409
Epoch 8/100
250/250 [==============================] - 0s 421us/step - loss: 0.3687 - accuracy: 0.8468
Epoch 9/100
250/250 [==============================] - 0s 418us/step - loss: 0.3602 - accuracy: 0.8505
Epoch 10/100
250/250 [==============================] - 0s 414us/step - l

250/250 [==============================] - 0s 424us/step - loss: 0.3319 - accuracy: 0.8640
Epoch 80/100
250/250 [==============================] - 0s 422us/step - loss: 0.3318 - accuracy: 0.8620
Epoch 81/100
250/250 [==============================] - 0s 426us/step - loss: 0.3321 - accuracy: 0.8625
Epoch 82/100
250/250 [==============================] - 0s 418us/step - loss: 0.3316 - accuracy: 0.8634
Epoch 83/100
250/250 [==============================] - 0s 416us/step - loss: 0.3316 - accuracy: 0.8636
Epoch 84/100
250/250 [==============================] - 0s 426us/step - loss: 0.3316 - accuracy: 0.8630
Epoch 85/100
250/250 [==============================] - 0s 422us/step - loss: 0.3319 - accuracy: 0.8631
Epoch 86/100
250/250 [==============================] - 0s 430us/step - loss: 0.3318 - accuracy: 0.8631
Epoch 87/100
250/250 [==============================] - 0s 426us/step - loss: 0.3315 - accuracy: 0.8648
Epoch 88/100
250/250 [==============================] - 0s 430us/step - loss:

Training Accuracy = 0.8646, loss function converged at 0.33

In [49]:
y_pred = ann.predict(X_test)
y_actual = np.array(y_test == 1).reshape(len(y_test), 1)
y_pred = (y_pred > 0.5)

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [53]:
from sklearn.metrics import confusion_matrix, accuracy_score
print(confusion_matrix(y_pred,y_actual))
print('Test Accuracy = {}'.format(accuracy_score(y_pred, y_actual)))

[[1527  210]
 [  68  195]]
Test Accuracy = 0.861
